# SPACEL workflow (1/3): Deconvolution by Spoint on mouse brain ST dataset

July 2023

Dataset: 75 ST slices of mouse brain ([here](https://zenodo.org/record/8167488))

In [1]:
import os
os.environ['PYTORCH_DEVICE'] = 'cpu'
os.environ['PYTORCH_MPS_ENABLE'] = '0'
import torch
torch.device('cpu')  # Set default device to CPU



import pandas as pd
import scanpy as sc
import anndata
import os
from tqdm import tqdm 
import scanpy as sc
import numpy as np
import sys



## Load spatial transcriptomics data

In [28]:
import sys
sys.path.append('/Users/u0096223/Documents/GitHub/PVLabTime/Merscope')
data_dir = '/Users/u0096223/Baidu/Science/Data PVDH/MERSCOPE data/P5_mouse_brain_CBD_panel/Cells'
# anndata_filename = os.path.join(data_dir,'15-11-24_23-22_P5_mouse_brain_CBD_panel_Cells_annotated.h5ad')
# anndata_filename = os.path.join(data_dir,'Spoint_result_mouse_brain_st.h5ad')

fig_root = os.path.join(data_dir,'Figures_spatial_mapping_retroSeq_xu')

sc_files ='/Users/u0096223/Documents/Transcriptomics/P5_thalamocortical_240715/P5_thalamocortical_Xcount.h5ad'
spatial_file =os.path.join(data_dir,'15-11-24_23-22_P5_mouse_brain_CBD_panel_Cells_not_normalized.h5ad')

pn_trained_model= os.path.join(fig_root,'SPOINT_trained_model.h5')


The input data are anndata objects stored raw counts for scRNA-seq and ST. The scRNA-seq anndata must have cell type annotation in ``.obs``.

In [3]:
adata = sc.read(spatial_file)
scadata = sc.read_h5ad(sc_files)

In [4]:
scadata.obs

,projection,Sample,Age,cell_id,class_label,class_name,class_bootstrapping_probability,subclass_label,subclass_name,subclass_bootstrapping_probability,supertype_label,supertype_name,supertype_bootstrapping_probability,cluster_label,cluster_name,cluster_alias,cluster_bootstrapping_probability,cell_type
AAATGGATCGCGACAT-1,RL_proj,CPJ__2260e8__XHA001,P5,AAATGGATCGCGACAT-1,CS20230722_CLAS_18,18 TH Glut,1.00,CS20230722_SUBC_150,150 CM-IAD-CL-PCN Sema5b Glut,1.00,CS20230722_SUPT_0650,0650 CM-IAD-CL-PCN Sema5b Glut_2,1.00,CS20230722_CLUS_2639,2639 CM-IAD-CL-PCN Sema5b Glut_2,5098,1.00,0650 CM-IAD-CL-PCN Sema5b Glut_2
AACATTCAGTCAGCCG-1,RL_proj,CPJ__2260e8__XHA001,P5,AACATTCAGTCAGCCG-1,CS20230722_CLAS_19,19 MB Glut,1.00,CS20230722_SUBC_159,159 IF-RL-CLI-PAG Foxa1 Glut,1.00,CS20230722_SUPT_0689,0689 IF-RL-CLI-PAG Foxa1 Glut_5,1.00,CS20230722_CLUS_2780,2780 IF-RL-CLI-PAG Foxa1 Glut_5,2362,1.00,0689 IF-RL-CLI-PAG Foxa1 Glut_5
AACCAATTCGCGTAAC-1,RL_proj,CPJ__2260e8__XHA001,P5,AACCAATTCGCGTAAC-1,CS20230722_CLAS_18,18 TH Glut,1.00,CS20230722_SUBC_150,150 CM-IAD-CL-PCN Sema5b Glut,1.00,CS20230722_SUPT_0650,0650 CM-IAD-CL-PCN Sema5b Glut_2,0.62,CS20230722_CLUS_2639,2639 CM-IAD-CL-PCN Sema5b Glut_2,5098,0.62,0650 CM-IAD-CL-PCN Sema5b Glut_2
AACCACGCATCGTACC-1,RL_proj,CPJ__2260e8__XHA001,P5,AACCACGCATCGTACC-1,CS20230722_CLAS_18,18 TH Glut,1.00,CS20230722_SUBC_147,147 AD Serpinb7 Glut,1.00,CS20230722_SUPT_0641,0641 AD Serpinb7 Glut_1,1.00,CS20230722_CLUS_2613,2613 AD Serpinb7 Glut_1,5020,1.00,0641 AD Serpinb7 Glut_1
AACTCAACAAGTATGC-1,RL_proj,CPJ__2260e8__XHA001,P5,AACTCAACAAGTATGC-1,CS20230722_CLAS_18,18 TH Glut,1.00,CS20230722_SUBC_147,147 AD Serpinb7 Glut,1.00,CS20230722_SUPT_0641,0641 AD Serpinb7 Glut_1,1.00,CS20230722_CLUS_2613,2613 AD Serpinb7 Glut_1,5020,1.00,0641 AD Serpinb7 Glut_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGTATTGGTTAGGTCT-1,PM_proj_only,CPJ__77d3a6__XHA002,P5,TGTATTGGTTAGGTCT-1,CS20230722_CLAS_18,18 TH Glut,1.00,CS20230722_SUBC_147,147 AD Serpinb7 Glut,1.00,CS20230722_SUPT_0641,0641 AD Serpinb7 Glut_1,1.00,CS20230722_CLUS_2613,2613 AD Serpinb7 Glut_1,5020,1.00,0641 AD Serpinb7 Glut_1
TGTCTGTTCAGTACCA-1,PM_proj_only,CPJ__77d3a6__XHA002,P5,TGTCTGTTCAGTACCA-1,CS20230722_CLAS_18,18 TH Glut,1.00,CS20230722_SUBC_151,151 TH Prkcd Grin2c Glut,1.00,CS20230722_SUPT_0658,0658 TH Prkcd Grin2c Glut_5,0.78,CS20230722_CLUS_2655,2655 TH Prkcd Grin2c Glut_5,5068,0.78,0658 TH Prkcd Grin2c Glut_5
TGTCTGTTCCTGTCTG-1,PM_proj_only,CPJ__77d3a6__XHA002,P5,TGTCTGTTCCTGTCTG-1,CS20230722_CLAS_13,13 CNU-HYa Glut,0.39,CS20230722_SUBC_117,117 LHA Barhl2 Glut,0.75,CS20230722_SUPT_0524,0524 LHA Barhl2 Glut_1,0.55,CS20230722_CLUS_2104,2104 LHA Barhl2 Glut_1,2743,0.55,0524 LHA Barhl2 Glut_1
TGTCTGTTCGCATTGA-1,PM_proj_only,CPJ__77d3a6__XHA002,P5,TGTCTGTTCGCATTGA-1,CS20230722_CLAS_14,14 HY Glut,0.92,CS20230722_SUBC_127,127 DMH-LHA Vgll2 Glut,1.00,CS20230722_SUPT_0560,0560 DMH-LHA Vgll2 Glut_2,1.00,CS20230722_CLUS_2286,2286 DMH-LHA Vgll2 Glut_2,1912,1.00,0560 DMH-LHA Vgll2 Glut_2


In [5]:
scadata.var_names_make_unique()
scadata.obs_names_make_unique()

## Initialize and train the Spoint model

In this step, we initialize the Spoint model using anndata objects for scRNA-seq and ST as input. The``celltype_key`` parameter represents the column name of the cell type annotation in the .obs attribute of the scRNA-seq anndata object. The ``sm_size`` parameter controls the number of simulated spots, and it is important to have a sufficient ``sm_size`` for accurate prediction. However, it should be noted that increasing the ``sm_size`` will also increase the simulation and training time. In general, we recommend setting ``sm_size`` to a value greater than 100,000.

In [6]:
import SPACEL
from SPACEL.setting import set_environ_seed
set_environ_seed()
from SPACEL import Spoint

Setting environment seed: 42


Global seed set to 0


In [8]:
spoint_model = Spoint.init_model(scadata,adata,celltype_key='cell_type',sm_size=500000,use_gpu=False,n_threads=2) #30min
spoint_model.train(max_steps=5000, batch_size=1024) # 11h!!!

Setting global seed: 42


/opt/anaconda3/envs/SPACEL/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:196: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))


### Finding marker genes...
### Filter cluster contain only one sample: ['0763 SCig Foxb1 Glut_4', '1154 CB Granule Glut_1', '0750 PAG Pou4f2 Mesi2 Glut_1', '0802 PAG-PPN Pax5 Sox21 Gaba_1', '0794 PPN-CUN-PCG Otp En1 Gaba_3', '0775 IC Tfap2d Maf Glut_6', '1156 DCO UBC Glut_1', '1189 VLMC NN_3', '0768 SCiw Pitx2 Glut_2', '0770 IC Tfap2d Maf Glut_1', '1173 Tanycyte NN_2', '0786 SCsg Pde5a Glut_2', '1163 Astro-TE NN_3', '1161 Astro-TE NN_1', '0773 IC Tfap2d Maf Glut_4', '0803 PAG-PPN Pax5 Sox21 Gaba_2', '1178 CHOR NN_1', '0964 NTS Phox2b Glut_6', '0811 IC Six3 En2 Gaba_5', '0992 MV-SPIV Phox2b Ebf3 Lbx1 Glut_1', '0866 PRT Mecom Gaba_3', '0745 PAG Pou4f2 Glut_1', '1031 CS-RPO Meis2 Gaba_1', '0887 SNc-VTA-RAmb Foxa1 Dopa_8', '1017 SPVO Mafa Meis2 Glut_1', '0997 MV-SPIV Zic4 Neurod2 Glut_1', '0898 PB Lmx1a Glut_4', '0901 PB Lmx1a Glut_7', '1037 PRNc Otp Gly-Gaba_2', '0907 LDT-PCG Vsx2 Lhx4 Glut_2', '0985 SPVI-SPVC Tlx3 Ebf3 Glut_5', '0915 PB Evx2 Glut_2', '0922 PB Evx2 Glut_9', '0983 SPVI-SP

Global seed set to 0
Global seed set to 0
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/SPACEL/lib/python3.8/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


Epoch 100/100: 100%|███████| 100/100 [19:26<00:00, 11.66s/it, loss=248, v_num=1]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|███████| 100/100 [19:26<00:00, 11.66s/it, loss=248, v_num=1]


Step 5000: Test inference loss=-0.566: 100%|█| 5000/5000 [11:10:52<00:00,  8.05s


In [ ]:
# save the model
torch.save(spoint_model.model.state_dict(), pn_trained_model)

In [ ]:
# to be tested, load spoint model
spoint_model.model.load_state_dict(torch.load(pn_trained_model))

Then, we utilize the trained model to predict the cell type composition of each spot in the spatial transcriptomics data. This prediction will generate a DataFrame object, where each row corresponds to a spot in the spatial transcriptomics data, each column represents a cell type from the single-cell RNA-seq data, and each entry indicates the proportion of a particular cell type in a spot. Additionally, we can obtain the anndata object of the spatial transcriptomics data with the deconvolution results embedded in the .obs attribute.

In [ ]:
# run prediction 
pre = spoint_model.deconv_spatial()
st_ad = spoint_model.st_ad

In [10]:
st_ad.write('/Users/u0096223/Baidu/Science/Data PVDH/MERSCOPE data/P5_mouse_brain_CBD_panel/Cells/Spoint_result_mouse_brain_st.h5ad')

In [19]:
import pandas as pd

# Save to CSV
pre.to_csv('/Users/u0096223/Baidu/Science/Data PVDH/MERSCOPE data/P5_mouse_brain_CBD_panel/Cells/spoint_model_deconv_spatial.csv', index=False)

## Visualization results

As a demonstration, we plotted the predicted compositions of cell types on a slice, where the cell types are selected as the top 20 with the highest proportion within each spot.

In [11]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.serif'] = ['Arial']
sc.settings.set_figure_params(dpi=50,dpi_save=300,facecolor='white',fontsize=10,vector_friendly=True,figsize=(3,3))
sc.settings.verbosity = 3

In [12]:
st_ad.obs

,Custom Cells groups,Datasets,volume,center_x,center_y,n_genes,batch,0641 AD Serpinb7 Glut_1,0650 CM-IAD-CL-PCN Sema5b Glut_2,0659 TH Prkcd Grin2c Glut_6,...,0591 PH-ant-LHA Otp Bsx Glut_1,0593 PH-ant-LHA Otp Bsx Glut_3,0594 PH-ant-LHA Otp Bsx Glut_4,0600 STN-PSTN Pitx2 Glut_3,0612 PH-LHA Foxb1 Glut_1,0614 PH-LHA Foxb1 Glut_3,0627 PH-SUM Foxa1 Glut_6,0630 MM Foxb1 Glut_1,0631 MM Foxb1 Glut_2,1191 Peri NN_1
4176488300003100058,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,679.729797,1864.685227,370.966180,1,real,0.113572,0.055957,0.150838,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4176488300003100061,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,421.822998,1844.958006,372.673463,1,real,0.121459,0.044972,0.207865,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4176488300003100074,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,875.621033,2026.338017,380.136557,1,real,0.171256,0.058806,0.132689,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4176488300003100075,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,2463.850586,1882.568016,386.052514,1,real,0.120751,0.064514,0.196988,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4176488300003100081,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_0,3240.593506,1721.412778,402.387977,1,real,0.105762,0.056548,0.217181,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4176488700197100074,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_1,2639.967285,5930.508174,12743.723030,1,real,0.120266,0.064146,0.150097,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4176488700197100120,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_1,2153.245361,5901.349850,12811.844244,1,real,0.130050,0.077804,0.134894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4176488700197100126,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_1,1115.132446,5775.783258,12817.580085,1,real,0.149157,0.070730,0.154846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4176488700197100136,Unclustered,202411071245_Xu-P5-ms-brain_VMSC08602_region_1,1696.354370,5652.479659,12830.909431,1,real,0.171256,0.058806,0.132689,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
import sys
sys.path.append('/Users/u0096223/Documents/GitHub/PVLabTime/Merscope')
from MerscopeBrainAnalysis import MerscopeBrainAnalysis

data_dir = '/Users/u0096223/Baidu/Science/Data PVDH/MERSCOPE data/P5_mouse_brain_CBD_panel/Cells'
fig_root = os.path.join(data_dir,'Figures_spatial_mapping_retroSeq_xu')

# sc_files ='/Users/u0096223/Documents/Transcriptomics/P5_thalamocortical_240715/P5_thalamocortical_Xcount.h5ad'
spatial_file =os.path.join(fig_root,'Spoint_result_mouse_brain_st.h5ad')

st_ad = anndata.read_h5ad(spatial_file)

ModuleNotFoundError: No module named 'cellphonedb'

In [14]:
# st_ad = st_ad[st_ad.obs.slice==37]
# celltype = st_ad.obs.loc[:,pre.columns].max(0)
st_ad = st_ad[st_ad.obs.Datasets.str.contains('region_0')]
celltype = st_ad.obs.loc[:,pre.columns].max(0)

In [37]:
print(st_ad.obs.columns.tolist())

['Custom Cells groups', 'Datasets', 'volume', 'center_x', 'center_y', 'n_genes', 'batch', '0641 AD Serpinb7 Glut_1', '0650 CM-IAD-CL-PCN Sema5b Glut_2', '0659 TH Prkcd Grin2c Glut_6', '0670 RE-Xi Nox4 Glut_3', '0646 PVT-PT Ntrk1 Glut_4', '0668 RE-Xi Nox4 Glut_1', '0661 TH Prkcd Grin2c Glut_8', '0653 CM-IAD-CL-PCN Sema5b Glut_5', '0689 IF-RL-CLI-PAG Foxa1 Glut_5', '0974 PGRN-PARN-MDRN Hoxb5 Glut_2', '0658 TH Prkcd Grin2c Glut_5', '0652 CM-IAD-CL-PCN Sema5b Glut_4', '0675 PF Fzd5 Glut_2', '0673 MG-POL-SGN Nts Glut_2', '0724 SPA-SPFm-SPFp-POL-PIL-PoT Sp9 Glut_7', '0674 PF Fzd5 Glut_1', '0642 AV Col27a1 Glut_1', '0880 SNc-VTA-RAmb Foxa1 Dopa_1', '0721 SPA-SPFm-SPFp-POL-PIL-PoT Sp9 Glut_4', '0702 APN C1ql2 Glut_1', '0790 ND-INC Foxd2 Glut_1', '0530 SI-MA-LPO-LHA Skor1 Glut_1', '0518 MS-SF Bsx Glut_1', '0717 PRC-PAG Tcf7l2 Irx2 Glut_5', '0651 CM-IAD-CL-PCN Sema5b Glut_3', '0671 RE-Xi Nox4 Glut_4', '0713 PRC-PAG Tcf7l2 Irx2 Glut_1', '0720 SPA-SPFm-SPFp-POL-PIL-PoT Sp9 Glut_3', '0649 CM-IAD-CL

In [15]:
sc.pl.embedding(st_ad,color=celltype.sort_values(ascending=False)[:20].index,basis='spatial',ncols=5)

KeyError: "Could not find 'spatial' or 'X_spatial' in .obsm"